# Bundle Composition Debug Notebook

This notebook demonstrates how Amplifier bundles compose, specifically investigating why the systems-thinking bundle observers aren't loading.

## Problem Statement

We have:
- **Root bundle** (`bundle.md`): Defines `hooks-observations` with `observers: [security-auditor, code-quality]`
- **Child bundle** (`examples/systems-thinking.md`): Includes root, redefines `hooks-observations` with `observers: [systems-dynamics, second-order-effects, ...]`

Expected: Child's observers should REPLACE root's observers (deep merge, lists replace).

Actual: Only root's observers appear in the mount plan.

## Setup: Import Required Modules

In [1]:
import sys
from pathlib import Path
import json

# Add the foundation bundle to path
foundation_path = Path.home() / ".amplifier/cache"
foundation_dir = next(foundation_path.glob("amplifier-foundation-*"), None)

if foundation_dir and foundation_dir not in [Path(p) for p in sys.path]:
    sys.path.insert(0, str(foundation_dir))
    print(f"Added to path: {foundation_dir}")

from amplifier_foundation.bundle import Bundle
from amplifier_foundation.dicts.merge import deep_merge, merge_module_lists

print("✓ Imports successful")

Added to path: /home/payne/.amplifier/cache/amplifier-foundation-c909465861f9d6ce
✓ Imports successful


## Test 1: Deep Merge Behavior

First, let's verify how `deep_merge()` handles lists.

In [2]:
# Test deep_merge with lists
parent = {
    "config": {
        "observers": ["observer-a", "observer-b"],
        "timeout": 30,
    }
}

child = {
    "config": {
        "observers": ["observer-x", "observer-y"],
        "max_concurrent": 5,
    }
}

result = deep_merge(parent, child)

print("Parent:", json.dumps(parent, indent=2))
print("\nChild:", json.dumps(child, indent=2))
print("\nMerged:", json.dumps(result, indent=2))
print("\n✓ Lists are REPLACED, not merged")
print(f"  Expected: ['observer-x', 'observer-y']")
print(f"  Got:      {result['config']['observers']}")
assert result['config']['observers'] == ['observer-x', 'observer-y']

Parent: {
  "config": {
    "observers": [
      "observer-a",
      "observer-b"
    ],
    "timeout": 30
  }
}

Child: {
  "config": {
    "observers": [
      "observer-x",
      "observer-y"
    ],
    "max_concurrent": 5
  }
}

Merged: {
  "config": {
    "observers": [
      "observer-x",
      "observer-y"
    ],
    "timeout": 30,
    "max_concurrent": 5
  }
}

✓ Lists are REPLACED, not merged
  Expected: ['observer-x', 'observer-y']
  Got:      ['observer-x', 'observer-y']


## Test 2: Module List Merging

Test how `merge_module_lists()` handles the same module ID appearing in both parent and child.

In [3]:
# Test merge_module_lists
parent_modules = [
    {
        "module": "hooks-observations",
        "source": "git+https://example.com/observers",
        "config": {
            "observers": ["security-auditor", "code-quality"]
        }
    }
]

child_modules = [
    {
        "module": "hooks-observations",
        "source": "git+https://example.com/observers",
        "config": {
            "observers": ["systems-dynamics", "second-order-effects"]
        }
    }
]

merged = merge_module_lists(parent_modules, child_modules)

print("Parent modules:", json.dumps(parent_modules, indent=2))
print("\nChild modules:", json.dumps(child_modules, indent=2))
print("\nMerged modules:", json.dumps(merged, indent=2))
print("\n✓ Child's observers list should REPLACE parent's")
print(f"  Expected: ['systems-dynamics', 'second-order-effects']")
print(f"  Got:      {merged[0]['config']['observers']}")
assert merged[0]['config']['observers'] == ['systems-dynamics', 'second-order-effects']

Parent modules: [
  {
    "module": "hooks-observations",
    "source": "git+https://example.com/observers",
    "config": {
      "observers": [
        "security-auditor",
        "code-quality"
      ]
    }
  }
]

Child modules: [
  {
    "module": "hooks-observations",
    "source": "git+https://example.com/observers",
    "config": {
      "observers": [
        "systems-dynamics",
        "second-order-effects"
      ]
    }
  }
]

Merged modules: [
  {
    "module": "hooks-observations",
    "source": "git+https://example.com/observers",
    "config": {
      "observers": [
        "systems-dynamics",
        "second-order-effects"
      ]
    }
  }
]

✓ Child's observers list should REPLACE parent's
  Expected: ['systems-dynamics', 'second-order-effects']
  Got:      ['systems-dynamics', 'second-order-effects']


## Test 3: Bundle Composition Order

Create simple bundles and test composition order.

In [4]:
# Create a root bundle
root_bundle = Bundle(
    name="observers-root",
    version="1.0.0",
    hooks=[
        {
            "module": "hooks-observations",
            "source": "git+https://example.com/observers",
            "config": {
                "hooks": [{"trigger": "orchestrator:complete"}],
                "observers": [
                    {"observer": "security-auditor", "watch": []},
                    {"observer": "code-quality", "watch": []}
                ]
            }
        }
    ]
)

print("Root bundle hooks:", json.dumps(root_bundle.hooks, indent=2))
print(f"\nRoot observers: {[o['observer'] for o in root_bundle.hooks[0]['config']['observers']]}")

Root bundle hooks: [
  {
    "module": "hooks-observations",
    "source": "git+https://example.com/observers",
    "config": {
      "hooks": [
        {
          "trigger": "orchestrator:complete"
        }
      ],
      "observers": [
        {
          "observer": "security-auditor",
          "watch": []
        },
        {
          "observer": "code-quality",
          "watch": []
        }
      ]
    }
  }
]

Root observers: ['security-auditor', 'code-quality']


In [5]:
# Create a child bundle that includes the root
child_bundle = Bundle(
    name="systems-thinking",
    version="1.0.0",
    includes=["observers-root"],  # Includes root (will be resolved to bundle object)
    hooks=[
        {
            "module": "hooks-observations",
            "source": "git+https://example.com/observers",
            "config": {
                "hooks": [{"trigger": "orchestrator:complete"}],
                "observers": [
                    {"observer": "systems-dynamics", "watch": []},
                    {"observer": "second-order-effects", "watch": []},
                    {"observer": "leverage-points", "watch": []}
                ]
            }
        }
    ]
)

print("Child bundle hooks:", json.dumps(child_bundle.hooks, indent=2))
print(f"\nChild observers: {[o['observer'] for o in child_bundle.hooks[0]['config']['observers']]}")

Child bundle hooks: [
  {
    "module": "hooks-observations",
    "source": "git+https://example.com/observers",
    "config": {
      "hooks": [
        {
          "trigger": "orchestrator:complete"
        }
      ],
      "observers": [
        {
          "observer": "systems-dynamics",
          "watch": []
        },
        {
          "observer": "second-order-effects",
          "watch": []
        },
        {
          "observer": "leverage-points",
          "watch": []
        }
      ]
    }
  }
]

Child observers: ['systems-dynamics', 'second-order-effects', 'leverage-points']


## Test 4: Manual Composition

Compose the bundles manually and inspect the result.

In [6]:
# Compose: root first, then child (this is the order from registry._compose_includes)
# From registry.py line 658-663:
#   result = included_bundles[0]  # Start with first include
#   for included in included_bundles[1:]:
#       result = result.compose(included)
#   return result.compose(bundle)  # Compose the current bundle LAST

composed = root_bundle.compose(child_bundle)

print("Composed bundle hooks:", json.dumps(composed.hooks, indent=2))
print(f"\nComposed observers: {[o['observer'] for o in composed.hooks[0]['config']['observers']]}")
print("\n✓ After composition, which observers survived?")
print(f"  Expected: Child's observers (systems-dynamics, second-order-effects, leverage-points)")
print(f"  Got:      {[o['observer'] for o in composed.hooks[0]['config']['observers']]}")

Composed bundle hooks: [
  {
    "module": "hooks-observations",
    "source": "git+https://example.com/observers",
    "config": {
      "hooks": [
        {
          "trigger": "orchestrator:complete"
        }
      ],
      "observers": [
        {
          "observer": "systems-dynamics",
          "watch": []
        },
        {
          "observer": "second-order-effects",
          "watch": []
        },
        {
          "observer": "leverage-points",
          "watch": []
        }
      ]
    }
  }
]

Composed observers: ['systems-dynamics', 'second-order-effects', 'leverage-points']

✓ After composition, which observers survived?
  Expected: Child's observers (systems-dynamics, second-order-effects, leverage-points)
  Got:      ['systems-dynamics', 'second-order-effects', 'leverage-points']


## Test 5: Simulate the Actual Registry Flow

The registry composes includes FIRST, then composes the current bundle. Let's simulate that.

In [7]:
# Simulate _compose_includes behavior from registry.py
# The child bundle has: includes=["observers-root"]

# Step 1: Load included bundles (in this case, just root_bundle)
included_bundles = [root_bundle]

# Step 2: Compose includes together (if multiple)
result = included_bundles[0]  # Start with first
for included in included_bundles[1:]:
    result = result.compose(included)

print("After composing includes:")
print(f"  Observers: {[o['observer'] for o in result.hooks[0]['config']['observers']]}")

# Step 3: Compose the current bundle (child_bundle) LAST
final = result.compose(child_bundle)

print("\nAfter composing child bundle:")
print(f"  Observers: {[o['observer'] for o in final.hooks[0]['config']['observers']]}")
print("\n✓ This is what the registry returns")
print(f"  Expected: {['systems-dynamics', 'second-order-effects', 'leverage-points']}")
print(f"  Got:      {[o['observer'] for o in final.hooks[0]['config']['observers']]}")

After composing includes:
  Observers: ['security-auditor', 'code-quality']

After composing child bundle:
  Observers: ['systems-dynamics', 'second-order-effects', 'leverage-points']

✓ This is what the registry returns
  Expected: ['systems-dynamics', 'second-order-effects', 'leverage-points']
  Got:      ['systems-dynamics', 'second-order-effects', 'leverage-points']


## Test 6: What if Child Also Declares tool-observations?

The systems-thinking bundle declares BOTH hooks and tools modules.

In [8]:
# Root bundle with both tool and hook
root_with_tool = Bundle(
    name="observers-root",
    version="1.0.0",
    tools=[
        {
            "module": "tool-observations",
            "source": "git+https://example.com/observers#tool-observations",
            "config": {}
        }
    ],
    hooks=[
        {
            "module": "hooks-observations",
            "source": "git+https://example.com/observers#hooks-observations",
            "config": {
                "observers": [
                    {"observer": "security-auditor"},
                    {"observer": "code-quality"}
                ]
            }
        }
    ]
)

# Child bundle ALSO declares both (like systems-thinking.md does)
child_with_tool = Bundle(
    name="systems-thinking",
    version="1.0.0",
    tools=[
        {
            "module": "tool-observations",
            "source": "git+https://example.com/observers#tool-observations",
            "config": {}  # Same config
        }
    ],
    hooks=[
        {
            "module": "hooks-observations",
            "source": "git+https://example.com/observers#hooks-observations",
            "config": {
                "observers": [
                    {"observer": "systems-dynamics"},
                    {"observer": "second-order-effects"}
                ]
            }
        }
    ]
)

# Compose
composed_full = root_with_tool.compose(child_with_tool)

print("Tools after composition:", len(composed_full.tools), "module(s)")
print("  -", composed_full.tools[0]['module'])
print("\nHooks after composition:", len(composed_full.hooks), "module(s)")
print("  -", composed_full.hooks[0]['module'])
print("\nObservers in composed bundle:")
observers = [o['observer'] for o in composed_full.hooks[0]['config']['observers']]
print(f"  {observers}")
print("\n✓ Expected child to win (systems-dynamics, second-order-effects)")
print(f"  Result: {'PASS' if observers == ['systems-dynamics', 'second-order-effects'] else 'FAIL'}")

Tools after composition: 1 module(s)
  - tool-observations

Hooks after composition: 1 module(s)
  - hooks-observations

Observers in composed bundle:
  ['systems-dynamics', 'second-order-effects']

✓ Expected child to win (systems-dynamics, second-order-effects)
  Result: PASS


## Test 7: Load Actual Bundle Files

Load the real bundle.md and systems-thinking.md to see what's actually defined.

In [10]:
import yaml
import re

def parse_bundle_frontmatter(file_path):
    """Parse bundle markdown file to extract YAML frontmatter."""
    content = Path(file_path).read_text()
    pattern = r'^---\s*\n(.*?)\n---\s*\n'
    match = re.match(pattern, content, re.DOTALL)
    if match:
        return yaml.safe_load(match.group(1))
    return {}

# Load bundle.md
bundle_md_path = Path.cwd() / ".." / "bundle.md"
bundle_md_yaml = parse_bundle_frontmatter(bundle_md_path)

print("=== bundle.md ===")
print(f"Name: {bundle_md_yaml['bundle']['name']}")
print(f"Hooks modules: {len(bundle_md_yaml.get('hooks', []))}")
if bundle_md_yaml.get('hooks'):
    hooks_obs = next((h for h in bundle_md_yaml['hooks'] if h['module'] == 'hooks-observations'), None)
    if hooks_obs:
        observers = [o['observer'] for o in hooks_obs['config']['observers']]
        print(f"Observers: {observers}")

=== bundle.md ===
Name: observers
Hooks modules: 2
Observers: ['observers/security-auditor', 'observers/code-quality']


In [11]:
# Load systems-thinking.md
systems_md_path = Path.cwd() / ".." / "examples/systems-thinking.md"
systems_md_yaml = parse_bundle_frontmatter(systems_md_path)

print("=== examples/systems-thinking.md ===")
print(f"Name: {systems_md_yaml['bundle']['name']}")
print(f"Includes: {systems_md_yaml.get('includes', [])}")
print(f"Hooks modules: {len(systems_md_yaml.get('hooks', []))}")
if systems_md_yaml.get('hooks'):
    hooks_obs = next((h for h in systems_md_yaml['hooks'] if h['module'] == 'hooks-observations'), None)
    if hooks_obs:
        observers = [o['observer'] for o in hooks_obs['config']['observers']]
        print(f"Observers: {observers}")
        print(f"\n✓ These are the observers that SHOULD appear in the mount plan")

=== examples/systems-thinking.md ===
Name: systems-thinking
Includes: [{'bundle': 'git+https://github.com/microsoft/amplifier-foundation@main'}]
Hooks modules: 1
Observers: ['observers/systems-dynamics', 'observers/second-order-effects', 'observers/leverage-points', 'observers/bias-detector', 'observers/stakeholder-analyzer']

✓ These are the observers that SHOULD appear in the mount plan


## Test 8: Check Session Mount Plan

Extract the actual mount plan from the session to see what was composed.

In [13]:
# Load the session mount plan
session_id = "2a87ee87-a196-4350-96e1-6a0dc82b6808"
events_file = Path.home() / f".amplifier/projects/-data-repos-msft-amplifier-bundle-observers/sessions/{session_id}/events.jsonl"

if events_file.exists():
    print(f"✓ Found session events: {events_file}")
    # Find session:start:debug event (new sessions) or session:resume:debug (resumed sessions)
    with events_file.open() as f:
        for line in f:
            event = json.loads(line)
            if event.get('event') in ['session:start:debug', 'session:resume:debug']:
                mount_plan = event.get('data', {}).get('mount_plan', {})
                if mount_plan:
                    hooks = mount_plan.get('hooks', [])
                    hooks_obs = next((h for h in hooks if h.get('module') == 'hooks-observations'), None)
                    if hooks_obs:
                        observers = hooks_obs.get('config', {}).get('observers', [])
                        observer_names = [o.get('observer') for o in observers]
                        print(f"\nSession {session_id[:8]}...")
                        print(f"Event: {event['event']}")
                        print(f"Observers in mount plan: {observer_names}")
                        print(f"\n✗ PROBLEM: Expected systems-thinking observers!")
                        print(f"  Expected: ['observers/systems-dynamics', 'observers/second-order-effects', ...]")
                        print(f"  Got:      {observer_names}")
                        
                        if observer_names[0] == 'observers/security-auditor':
                            print(f"\n  ✗ Only seeing ROOT bundle observers (security-auditor, code-quality)")
                        elif 'systems-dynamics' in observer_names[0]:
                            print(f"\n  ✓ Seeing SYSTEMS-THINKING observers!")
                    else:
                        print(f"\n✗ No hooks-observations module found in mount plan")
                    break
else:
    print(f"✗ Session file not found: {events_file}")
    print(f"\nTry updating session_id to match your test session.")


## Test 9: Compose Order Reverse Test

What if we compose in reverse order? Let's verify the order matters.

In [ ]:
# Test composing in different orders
print("Order 1: root.compose(child)")
order1 = root_bundle.compose(child_bundle)
obs1 = [o['observer'] for o in order1.hooks[0]['config']['observers']]
print(f"  Result: {obs1}")
print(f"  {'✓ PASS' if 'systems-dynamics' in obs1 else '✗ FAIL'} - Child should win")

print("\nOrder 2: child.compose(root)")
order2 = child_bundle.compose(root_bundle)
obs2 = [o['observer'] for o in order2.hooks[0]['config']['observers']]
print(f"  Result: {obs2}")
print(f"  {'✓ PASS' if 'security-auditor' in obs2 else '✗ FAIL'} - Root should win")

print("\n✓ Composition order DOES matter")
print("  Last bundle wins for list-type config values")

## Test 10: Investigate Registry Composition Order

Check if the registry is somehow composing in the wrong order.

In [ ]:
# Simulate the exact registry flow from _compose_includes (line 658-663)
# For a bundle with includes=["observers-root"], the flow is:

current_bundle = child_bundle  # The bundle being loaded (systems-thinking)
included_bundles = [root_bundle]  # Its includes (observers root)

# Registry code:
# result = included_bundles[0]
# for included in included_bundles[1:]:
#     result = result.compose(included)
# return result.compose(bundle)  # ← Current bundle composes LAST

print("Step 1: Start with first include (root_bundle)")
result = included_bundles[0]
print(f"  Observers: {[o['observer'] for o in result.hooks[0]['config']['observers']]}")

print("\nStep 2: Compose remaining includes (none in this case)")
for included in included_bundles[1:]:
    result = result.compose(included)
print(f"  Observers: {[o['observer'] for o in result.hooks[0]['config']['observers']]}")

print("\nStep 3: Compose current bundle (systems-thinking) LAST")
result = result.compose(current_bundle)
print(f"  Observers: {[o['observer'] for o in result.hooks[0]['config']['observers']]}")

print("\n✓ Final composition result:")
final_observers = [o['observer'] for o in result.hooks[0]['config']['observers']]
print(f"  {final_observers}")
print(f"\n  Expected: systems-thinking observers")
print(f"  Match: {final_observers == ['systems-dynamics', 'second-order-effects', 'leverage-points']}")

## Test 11: Check for Circular Dependency Detection

What if the sub-bundle including its root triggers circular dependency detection?

In [ ]:
# The systems-thinking bundle is a subdirectory of the observers bundle
# URI: git+https://github.com/payneio/amplifier-bundle-observers@main#examples/systems-thinking.md
# It includes: git+https://github.com/payneio/amplifier-bundle-observers@main (the root)

# From registry.py lines 346-350:
# is_subdirectory = "#subdirectory=" in uri
# if not is_subdirectory and (uri in loading_chain or base_uri in loading_chain):
#     raise BundleDependencyError(f"Circular dependency detected: {uri}")

print("Scenario: Sub-bundle includes its own root")
print("\nSub-bundle URI:")
print("  git+https://.../observers@main#examples/systems-thinking.md")
print("\nRoot include:")
print("  git+https://.../observers@main")
print("\nCircular dependency check:")
print("  - Sub-bundle has fragment (#examples/...) → is_subdirectory = True")
print("  - Loading chain only includes full URI, not base_uri (line 366)")
print("  - Root bundle URI is different from sub-bundle URI")
print("  - ✓ Should NOT trigger circular dependency error")
print("\nConclusion: Not a circular dependency issue")

## Test 12: The Real Problem - Sub-bundle Including Root

Let's trace what happens when systems-thinking.md (a sub-bundle) includes its root.

In [ ]:
# When loading systems-thinking.md, the registry:
# 1. Detects it's a sub-bundle (has #subdirectory in URI)
# 2. Walks up to find root bundle.md
# 3. Loads the root bundle as metadata (for namespace resolution)
# 4. Then processes includes from systems-thinking.md

# The includes list is: ["foundation", "observers@main"]
# When it tries to load "observers@main", does it:
#   A) Load the root bundle AGAIN (duplicate load)
#   B) Detect it's already loaded and use cache
#   C) Something else?

print("Hypothesis: Root bundle gets loaded TWICE")
print("\nLoad sequence for systems-thinking.md:")
print("  1. Load systems-thinking.md (the file)")
print("  2. Detect root bundle.md above it (for namespace)")
print("  3. Process includes: ['foundation', 'observers@main']")
print("  4. Load 'observers@main' → This loads bundle.md AGAIN")
print("  5. Compose: foundation → observers_root → systems_thinking")
print("\nProblem scenario:")
print("  If bundle.md is loaded as the root bundle for namespace,")
print("  AND loaded again via includes,")
print("  The second load might use cached version with ORIGINAL config.")
print("\n✓ This would explain why root's observers survive!")

## Test 13: Solution Verification

Verify that removing the include fixes the issue.

In [ ]:
# Child bundle WITHOUT including the root
child_standalone = Bundle(
    name="systems-thinking",
    version="1.0.0",
    includes=[],  # No includes!
    tools=[
        {
            "module": "tool-observations",
            "source": "git+https://example.com/observers#tool",
        }
    ],
    hooks=[
        {
            "module": "hooks-observations",
            "source": "git+https://example.com/observers#hook",
            "config": {
                "observers": [
                    {"observer": "systems-dynamics"},
                    {"observer": "second-order-effects"},
                    {"observer": "leverage-points"}
                ]
            }
        }
    ]
)

print("Standalone child bundle (no includes):")
observers = [o['observer'] for o in child_standalone.hooks[0]['config']['observers']]
print(f"  Observers: {observers}")
print(f"\n✓ With no includes, child's config is authoritative")
print(f"  This is the fix we applied to examples/systems-thinking.md")

## Conclusion

### What We Learned

1. **Deep merge works correctly** - Lists DO replace (child wins)
2. **Module merging works correctly** - Same module ID deep merges configs
3. **Composition order works correctly** - Last bundle wins

### The Bug

The issue is **sub-bundle including its own root creates a complex scenario**:
- The root bundle loads for namespace resolution
- The root bundle loads AGAIN via includes
- Caching or loading order causes the root's config to persist

### The Fix

**Remove the self-reference include** from examples:
```yaml
# ✗ BEFORE
includes:
  - bundle: git+https://.../foundation@main
  - bundle: git+https://.../observers@main  # Self-reference!

# ✓ AFTER
includes:
  - bundle: git+https://.../foundation@main
  # No observers include - declare modules directly
```

This makes each example bundle **self-contained** with its own module declarations.

## Test 14: Verify Cached Bundle Has Self-Reference Removed

Check if the cached bundles from GitHub have the fix applied.

In [ ]:
# Check all cached versions of the observers bundle
cache_root = Path.home() / ".amplifier/cache"
print("Checking cached bundle versions:\n")

for cache_dir in sorted(cache_root.glob("amplifier-bundle-observers-*")):
    st_file = cache_dir / "examples/systems-thinking.md"
    if st_file.exists():
        content = st_file.read_text()
        
        # Check if self-reference exists in includes section
        includes_section = content.split('hooks:')[0] if 'hooks:' in content else content
        has_self_ref = "payneio/amplifier-bundle-observers@main" in includes_section
        
        # Also check what observers are defined
        has_systems_dynamics = "observers/systems-dynamics" in content
        
        print(f"Cache: {cache_dir.name}")
        print(f"  Self-reference in includes: {has_self_ref}")
        print(f"  Has systems-dynamics observer: {has_systems_dynamics}")
        
        if not has_self_ref and has_systems_dynamics:
            print(f"  ✓ GOOD: Fix is present in cache")
        elif has_self_ref:
            print(f"  ✗ BAD: Still has self-reference (old cached version)")
        print()


## Test 15: The Duplicate Hooks Bug

The mount plan shows TWO hooks-observations modules! This shouldn't happen.

In [ ]:
# Load the actual session mount plan and check for duplicates
session_id = "2a87ee87-a196-4350-96e1-6a0dc82b6808"
events_file = Path.home() / f".amplifier/projects/-data-repos-msft-amplifier-bundle-observers/sessions/{session_id}/events.jsonl"

if events_file.exists():
    with events_file.open() as f:
        for line in f:
            event = json.loads(line)
            if event.get('event') == 'session:start:debug':
                mount_plan = event.get('data', {}).get('mount_plan', {})
                hooks = mount_plan.get('hooks', [])
                
                # Find ALL hooks-observations modules
                hooks_obs_modules = [h for h in hooks if h.get('module') == 'hooks-observations']
                
                print(f"Total hooks in mount plan: {len(hooks)}")
                print(f"hooks-observations modules: {len(hooks_obs_modules)}")
                print()
                
                for i, hook in enumerate(hooks_obs_modules):
                    observers = hook.get('config', {}).get('observers', [])
                    observer_names = [o.get('observer') for o in observers]
                    print(f"Instance {i+1}:")
                    print(f"  Source: {hook.get('source', 'N/A')}")
                    print(f"  Observers ({len(observers)}): {observer_names}")
                    print()
                
                print("✗ BUG CONFIRMED: hooks-observations appears TWICE!")
                print("  merge_module_lists should deduplicate by module ID")
                print("\nThis means:")
                print("  1. Bundle composition isn't deduplicating properly")
                print("  2. Both instances have ROOT observers (not systems-thinking)")
                print("  3. This is why systems-thinking observers never load")
                break
else:
    print(f"Session not found: {events_file}")


## Test 16: Load Through Actual BundleRegistry

Load the systems-thinking bundle using the actual registry to see what it returns.

In [ ]:
from amplifier_foundation.registry import BundleRegistry
import asyncio

async def load_and_inspect():
    registry = BundleRegistry()
    
    # Load systems-thinking bundle the way the session does
    print("Loading systems-thinking bundle through registry...\n")
    bundle = await registry.load("systems-thinking")
    
    print(f"Bundle name: {bundle.name}")
    print(f"Bundle version: {bundle.version}")
    print(f"Includes: {bundle.includes}")
    print(f"\nHooks modules: {len(bundle.hooks)}")
    
    # List all hooks modules
    for i, hook in enumerate(bundle.hooks):
        print(f"  {i+1}. {hook.get('module')}")
    
    # Find hooks-observations
    hooks_obs_list = [h for h in bundle.hooks if h.get('module') == 'hooks-observations']
    print(f"\nhooks-observations instances: {len(hooks_obs_list)}")
    
    for i, hooks_obs in enumerate(hooks_obs_list):
        observers = hooks_obs.get('config', {}).get('observers', [])
        observer_names = [o.get('observer') for o in observers]
        print(f"\nInstance {i+1}:")
        print(f"  Observers ({len(observers)}): {observer_names}")
        
        if observer_names and 'systems-dynamics' in observer_names[0]:
            print(f"  ✓ CORRECT: Systems-thinking observers loaded")
        elif observer_names and 'security-auditor' in observer_names[0]:
            print(f"  ✗ WRONG: Root bundle observers (should be systems-thinking)")
    
    return bundle

bundle = asyncio.run(load_and_inspect())


## Test 17: The Subdirectory Bundle Bug

Test 16 revealed: `registry.load("systems-thinking")` returns bundle with `name="observers"` (the ROOT)!

This is the bug - subdirectory bundles are returning their root instead of themselves.

In [ ]:
from amplifier_foundation.registry import BundleRegistry
import asyncio

async def trace_subdirectory_load():
    registry = BundleRegistry()
    
    # Load by registered name
    print("Test 1: Load by registered name 'systems-thinking'")
    bundle1 = await registry.load("systems-thinking")
    print(f"  Returned bundle name: {bundle1.name}")
    print(f"  Expected: 'systems-thinking'")
    print(f"  Match: {bundle1.name == 'systems-thinking'}")
    print()
    
    # Load by full URI
    print("Test 2: Load by full URI (with #examples/...)")
    uri = "git+https://github.com/payneio/amplifier-bundle-observers@main#examples/systems-thinking.md"
    bundle2 = await registry._load_single(uri, auto_register=False, auto_include=True)
    print(f"  Returned bundle name: {bundle2.name}")
    print(f"  Expected: 'systems-thinking'")
    print(f"  Match: {bundle2.name == 'systems-thinking'}")
    print()
    
    # Check observers in both
    print("Test 3: Check which observers each returned")
    for label, bundle in [("Load by name", bundle1), ("Load by URI", bundle2)]:
        hooks_obs = next((h for h in bundle.hooks if h.get('module') == 'hooks-observations'), None)
        if hooks_obs:
            observers = [o.get('observer') for o in hooks_obs.get('config', {}).get('observers', [])]
            print(f"  {label}: {observers[:2]}...")  # First 2
    
    return bundle1, bundle2

result = asyncio.run(trace_subdirectory_load())
